# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

****

In [14]:
import pandas as pd
import numpy as np
from scipy import stats

In [15]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [16]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [17]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [18]:
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [19]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [20]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [21]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit



<p>Your answers to Q1 and Q2 here</p>

** Q1: ** The appropriate test is an A/B test.<br>The CLT does apply because the resumes were randomly assigned, there are more than 30 observations and this is a binomial distribution.<br><br>
**Q2:  **  H0 = There is no statistically significant difference in call backs for white sounding names and black sounding names.<br>
H1 = There is a statistically significant difference in call backs for white sounding names and black sounding names.

In [22]:
w = data[data.race=='w']
b = data[data.race=='b']

In [23]:
# Frequentist Model for Q3

crit_val = 1.96
mdiff = np.mean(w.call)-np.mean(b.call)
se = np.std(w.call)/np.sqrt(len(w.call)) + np.std(b.call)/len(b.call)
moe = se*crit_val
ci = [mdiff-moe,mdiff+moe]
z = mdiff/se
p = stats.norm.cdf(1-z)

print('Margin of Error: ',moe,' Confidence Interval: ',ci,' Z Score: ',z,' P: ',p)

Margin of Error:  0.0119263328859  Confidence Interval:  [0.020106521969160015, 0.043959187740961136]  Z Score:  5.26435041824  P:  1.00242298928e-05


In [24]:
# Bootstrapping Model for Q3
# Creating Function to Calculate the Difference Fracton of the Two Data Sets

def diff_frac(data1,data2):
    '''Calculates the difference in fractions between two bernoulli data sets.'''
    frac1 = np.sum(data1)/ len(data1)
    frac2 = np.sum(data2)/ len(data2)
    return frac2 - frac1

diff_frac_bw = diff_frac(b.call, w.call)


In [25]:
# Bootstrap Data
# Confidence Intervals
b_reps = draw_bs_reps(b.call, np.mean, 10000)
w_reps = draw_bs_reps(w.call, np.mean, 10000)
bci= np.percentile(b_reps,[2.5,97.5])
wci= np.percentile(w_reps,[2.5,97.5])

# Margin of Error
bmoe_bs = (np.std(b_reps)/np.sqrt(len(b_reps)))*crit_val
wmoe_bs = (np.std(w_reps)/np.sqrt(len(w_reps)))*crit_val

# P Value of A/B Test
perm_reps = draw_perm_reps(b.call, w.call, diff_frac, 10000)
p = np.sum(perm_reps >= diff_frac_bw)/10000
print('Black Sounding Names: Confidence Interval: ', bci,'\n','Margin of Error: ',bmoe_bs,'\n',
     'White Sounding Names: Confidence Interval: ',wci, '\n', 'Margin of Error: ',wmoe_bs,'\n'
     'P Value: ',p)

Black Sounding Names: Confidence Interval:  [ 0.0550308   0.07433265] 
 Margin of Error:  9.76978749318e-05 
 White Sounding Names: Confidence Interval:  [ 0.08501027  0.10800821] 
 Margin of Error:  0.000115726862144 
P Value:  0.0


<p> Your answers to Q4 and Q5 here </p>

**Q4: **  This analysis indicates that there is a statistically significant difference in the number of call-backs for black sounding names vs white sounding names.  Since the total number of call-backs for white sounding names is higher, one can infer that black sounding names are called back significantly lower than white sounding names. As such, the data indicates that racial bias continues to exist within the US labor market.<br>
**Q5: **  From this analysis we cannot tell if racially specific sounding names play the most important factor in call-back success.  It is very possible that other factors such as years of experience, number of jobs, or other variables listed in the data play a more significant role.  I would amend this study by performing a multivariate A/B test to include these other factors to determine how they stack up to diverse sounding names.